In [1]:
import geopandas as gpd

path = "/workspaces/Satelite/data/CH.gpkg"
gpd = gpd.read_file(path)

In [4]:
# See all features:
print(gpd.head(100))

    bezugsjahr  lnf_code       nutzung  ist_ueberlagernd        code_programm  \
0         2023       501  Sommergerste             False                  Non   
1         2023       501  Sommergerste             False                  Non   
2         2023       501  Sommergerste             False                  Non   
3         2023       501  Sommergerste             False                  Non   
4         2023       501  Sommergerste             False                  Non   
..         ...       ...           ...               ...                  ...   
95        2023       502  Wintergerste             False                  Bio   
96        2023       502  Wintergerste             False  psSoilMildTreatment   
97        2023       502  Wintergerste             False  psSoilMildTreatment   
98        2023       502  Wintergerste             False  psSoilMildTreatment   
99        2023       502  Wintergerste             False                  Non   

                           

In [5]:
nutzung = gpd['nutzung'].unique()

In [8]:
to_remove = [
            "Buntbrache",
            "Rotationsbrache",
            "Saum auf Ackerflächen",
            "Nützlingsstreifen auf offener Ackerfläche",
            "Übrige offene Ackerfläche, nicht beitragsberechtigt (regionsspezifische Biodiversitätsförderfläche)",
            "Christbäume",
            "Hecken-, Feld- und Ufergehölze (mit Krautsaum)",
            "Hecken-, Feld- und Ufergehölze (mit Pufferstreifen)",
            "Wald",
            "Übrige unproduktive Flächen (z.B. gemulchte Flächen, stark verunkrautete Flächen, Hecken ohne Pufferstreifen)",
            "Wassergräben, Tümpel, Teiche",
            "Ruderalflächen, Steinhaufen und -wälle",
            "Unbefestigte, natürliche Wege",
            "Regionsspezifische Biodiversitätsförderflächen",
            "Hausgärten",
            "Sömmerungsweiden",
            "Übrige Flächen ausserhalb der LN und SF",
            "Flächen ohne landwirtschaftliche Hauptzweckbestimmung (erschlossenes Bauland, Spiel-, Reit-, Camping-, Golf-, Flug- und Militärplätze oder ausgemarchte Bereiche von Eisenbahnen, öffentlichen Strassen und Gewässern)",
            "Offene Ackerfläche, beitragsberechtigt (regionsspezifische Biodiversitätsförderfläche)",
            "Waldweiden (ohne bewaldete Fläche)",
            "Heuwiesen im Sömmerungsgebiet, Übrige Wiesen",
            "Einheimische standortgerechte Einzelbäume und Alleen (Punkte oder Flächen)",
            "Andere Bäume",
            "Andere Bäume (regionsspezifische Biodiversitätsförderfläche)",
            "Andere Elemente (regionsspezifische Biodiversitätsförderfläche)","Quinoa",
            "Heuwiesen im Sömmerungsgebiet, Typ extensiv genutzte Wiese",
            "Heuwiesen im Sömmerungsgebiet, Typ wenig intensiv genutzte Wiese",
            "Hecken-, Feld- und Ufergehölze (mit Pufferstreifen) (regionsspezifische Biodiversitätsförderfläche)",
            "Trockenmauern",
            "Regionsspezifische Biodiversitätsförderflächen (Weiden)",
            "Baumschule von Forstpflanzen ausserhalb der Forstzone",
            "Ackerschonstreifen",
            "Landwirtschaftliche Produktion in Gebäuden (z. B. Champignon, Brüsseler)",
            "Heuwiesen mit Zufütterung während der Sömmerung",
            "Streueflächen im Sömmerungsgebiet",
            "Regionsspezifische Biodiversitätsförderfläche (Grünflächen ohne Weiden)"
    
        ]

print(len(to_remove))


# Check if the values are in the list
count = 0
for i in nutzung:
    if i in to_remove:
        count += 1
        print(i)
        gpd = gpd[gpd.nutzung != i]

159
Buntbrache
Rotationsbrache
Saum auf Ackerflächen
Nützlingsstreifen auf offener Ackerfläche
Übrige offene Ackerfläche, nicht beitragsberechtigt (regionsspezifische Biodiversitätsförderfläche)
Regionsspezifische Biodiversitätsförderfläche (Grünflächen ohne Weiden)
Christbäume
Hecken-, Feld- und Ufergehölze (mit Krautsaum)
Hecken-, Feld- und Ufergehölze (mit Pufferstreifen)
Wald
Übrige unproduktive Flächen (z.B. gemulchte Flächen, stark verunkrautete Flächen, Hecken ohne Pufferstreifen)
Wassergräben, Tümpel, Teiche
Ruderalflächen, Steinhaufen und -wälle
Unbefestigte, natürliche Wege
Regionsspezifische Biodiversitätsförderflächen
Hausgärten
Sömmerungsweiden
Übrige Flächen ausserhalb der LN und SF
Flächen ohne landwirtschaftliche Hauptzweckbestimmung (erschlossenes Bauland, Spiel-, Reit-, Camping-, Golf-, Flug- und Militärplätze oder ausgemarchte Bereiche von Eisenbahnen, öffentlichen Strassen und Gewässern)
Streueflächen im Sömmerungsgebiet
Offene Ackerfläche, beitragsberechtigt (regio

In [18]:
import pandas as pd
unique_nutzung = gpd['nutzung'].unique()

# Convert the unique values to a DataFrame and add an index
df_unique_nutzung = pd.DataFrame(unique_nutzung, columns=['nutzung'])
df_unique_nutzung.reset_index(inplace=True)
df_unique_nutzung.rename(columns={'index': 'id'}, inplace=True)

# Save the DataFrame to a CSV file
output_path = "/workspaces/Satelite/data/unique_nutzung.csv"
df_unique_nutzung.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"CSV file with unique values has been created at: {output_path}")

    

CSV file with unique values has been created at: /workspaces/Satelite/data/unique_nutzung.csv


In [ ]:
# Save the data as a .csv file
